# This file contains functions used to generate new schedules when add/remove routes

+ **get_trips_for_routes**
  - From trips.txt, get trip_id for certain routes
+ **filter_stoptimes_by_routes**
  - Keep only rows that are related to certain routes in stop_times.txt
+ **convert_time_to_sec**
  - For arrival_time/departure_time in stop_time.txt, HH:MM:SS -> HH*60*60+MM*60+SS
+ **time_to_sec**
  - For single string time in format "08:00:00", convert to seconds
+ **get_travel_time**
  - Based on the existing schedule, get the travel time and distance for sequenced two stops, return in a pd dataframe
+ **get_total_travel_time**
  - Get total travel time for a new routes based on existing rotues' information
+ **gen_new_schedule**
  - generate a new stop_time like schedule

...
    
## Some settings/configurations

1. **trip_id** for new schedules are 6-digits number starts with doubel 9. ie. "99xxxx"
2. 

In [1]:
import pandas as pd

In [2]:
def get_trips_for_routes(routes = [30,34,36]):
    ### GET TRIPS FOR CERTAIN ROUTES ###
    # para: routes in list (float number)
    # return: trip_id mapping with rotue numbers in dictionary
    
    trips_dict = dict() # A dictionary mapping routes with multiple trips
    
    # read trips.txt file, keep only 2 columns of routes_short_name and trip_id
    trips = (pd.read_csv(r"mmt_gtfs/trips.txt", sep = ",", engine = "python")
            [['route_short_name','trip_id']])
    
    # for each routes (30,34,36), select their trips and update into stop_dict
    for r in routes:
        trip_id = trips.loc[trips['route_short_name'] == r,:]
        tp = trip_id["trip_id"].tolist()
        
        trips_dict.update({r:tp})

    return(trips_dict)

In [3]:
def filter_stoptimes_by_routes(routes = [30.,34,36]):
    ### READ stop_times.txt, AND FIILTER WITH CERTIAN ROUTES ###
    # para: routes in a list
    # return: filtered stop_times in pd.dataframe
    # calling function: get_trips_for_routes
    
    # get trips into a list
    tr_dict = get_trips_for_routes(routes)
    
    tr_list = []
    for r,tr in tr_dict.items():
        tr_list += tr
    
    # read stoptimes.txt file
    stop_times = pd.read_csv(r"mmt_gtfs/stop_times.txt", sep = ",", engine = "python")
    # filter by trips under certain routes
    stop_times_filter = stop_times.loc[stop_times['trip_id'].isin(tr_list)].reset_index()

    # return filtered stop_times in dataframe    
    return stop_times_filter
    

In [4]:
def convert_time_to_sec(stop_times):
    ###Convert str arrival_time/departure_time to int seconds###
    # HH:MM:SS -> HH*60*60+MM*60+SS
    # para: stop_times (a standard GTFS stop_times file)(pd dataframe)
    # return: stop_times with two columns added on
   
    df = stop_times
    
    # split string time into hours, minutes, and seconds
    df[['ar_h','ar_m','ar_s']] = df.arrival_time.str.split(':',expand=True)
    df[['dp_h','dp_m','dp_s']] = df.departure_time.str.split(':',expand=True)

    # compute time into seconds
    df=(df.assign(arrival_time = lambda x: df['ar_h'].astype(int)*60*60+df['ar_m'].astype(int)*60+df['ar_s'].astype(int),
                departure_time = lambda x: df['dp_h'].astype(int)*60*60+df['dp_m'].astype(int)*60+df['dp_s'].astype(int))
        .filter(regex = '^((?!ar_).)*$')  # remove intermediate variables
        .filter(regex = '^((?!dp_).)*$'))
    

    return df


def convert_time_to_str(stop_times_sec):
    
    ### Incomplete Function...###
    
    #FIXEME: intend to be the invert function of convert_time_to_sec
    
    df = stop_times
    df = df.assign(ar_h = lambda x: df['arrival_time']//3600,
                    ar_m = lambda x: df['arrival_time']%3600//60,
                    ar_s = lambda x: df['arrival_time']%3600%60,
                  dp_h = lambda x: df['departure_time']//3600,
                    dp_m = lambda x: df['departure_time']%3600//60,
                    dp_s = lambda x: df['departure_time']%3600%60)
    
def time_to_sec(time = '8:00:00'):
    
    ### Convert single string time into int seconds ###
    # HH:MM:SS -> HH*60*60+MM*60+SS
    # para: time 'HH:MM:SS'
    # return: time in seconds
    
    t_sec = int(time[0:-6])*60*60 + int(time[-5:-3])*60 + int(time[-2:-1])
    
    return t_sec

In [5]:
def get_travel_time(stop_list, stop_times):
    ### Based on the existing schedule, get the travel time and distance for sequenced two stops, return in a pd dataframe###
    # para: stop_list (a list of existing stop_id)
    #       stop_times (an existing schedule, arrival_times MUST BE CONVERTED TO SECONDS)
    # return: stop_seq (pd dataframe)
    
    # Create a new df to store outputs
    stop_seq = pd.DataFrame({"upstop_id" : [],
                             "downstop_id":[],
                             "travel_time" : [],
                             "travel_dist" : []})
    
    # for every two sequenced stop_ids in the given stop_list, search for the matching result in stop_times
    for i in range(len(stop_list)-1):
        # get a pair of stop_ids 
        upstop = stop_list[i]
        downstop = stop_list[i+1]

        # search in the existing schedule
        for j in range(len(stop_times)-1):
            if stop_times['stop_id'][j] == upstop:
                if stop_times['stop_id'][j+1] == downstop:
                    if stop_times['stop_sequence'][j+1]-stop_times['stop_sequence'][j] == 1:
                        
                        # get travel time and distance
                        time = stop_times['arrival_time'][j+1]-stop_times['arrival_time'][j]
                        dist = stop_times['shape_dist_traveled'][j+1]-stop_times['shape_dist_traveled'][j]
                        
                        # append to stop_seq the dataframe 
                        row={'upstop_id':upstop,'downstop_id':downstop,'travel_time':time,'travel_dist':dist}
                        stop_seq.loc[i]=row

                        # for every pair (upstop,downstop), once we got a result, move to the next pair
                        break

    return stop_seq

def get_total_travel_time(stop_seq):
    
    total_travel_time = sum(stop_seq.travel_time)
    total_travel_dist = sum(stop_seq.travel_dist)
    
    return total_travel_time, total_travel_dis
    

In [8]:
def get_available_tripid(stop_times):
    
    a = stop_times.max().trip_id
    if a >= 990000:
        b = a+1
    else:
        b = 990000
    
    return b

def gen_new_schedule(stop_list = [],
                     first_departure_time = '8:00:00',
                     last_departure_time = '18:00:00',
                     interval_time = 60, 
                     stop_seq = pd.DataFrame(),
                     stop_times_old = pd.DataFrame()):
    
    # convert time to second
    f_t = time_to_sec(first_departure_time)
    l_t = time_to_sec(last_departure_time)
    i_t = interval_time * 60

    # do the math
    stop_num = len(stop_list) # total stops per trip
    trip_num = int((l_t-f_t)/i_t) # total trips per day
    tr_id_1 = get_available_tripid(stop_times_old) # get the first available trip_id (avoid duplication)

    # create an empty dataframe 
    st_tm = pd.DataFrame({'trip_id':[],
                          'stop_sequence':[],
                          'stop_id':[],
                          'pickup_type':[],
                          'drop_off_type':[],
                          'arrival_time':[],
                          'departure_time':[],
                          'timepoint':[],
                          'stop_headsign':[],
                          'shape_dist_traveled':[]})

    for i in range(trip_num):

        tr_id = tr_id_1 + i # trip_id

        # get baseline for travel time/distance
        time_base = f_t + i * i_t
        dist_base = 0

        for j,st in enumerate(stop_list):


            # get arrival time and distance. if the stop is the first stop, travel_time/dist = 0
            if j == 0:
                travel_time = 0
                travel_dist = 0
            else:
                up_st = stop_list[j-1]
                travel_time = stop_seq.loc[stop_seq['downstop_id']==st,:].loc[stop_seq['upstop_id']==up_st,:].reset_index().at[0,'travel_time']
                travel_dist = stop_seq.loc[stop_seq['downstop_id']==st,:].loc[stop_seq['upstop_id']==up_st,:].reset_index().at[0,'travel_dist']
            
            time_base += travel_time 
            dist_base += travel_dist

            # assign pickup_type/drop_off_type
            pk_type = 0 # regularly scheduled pickup
            dr_type = 0 # regularly scheduled drop off
            if j == 0:
                dr_type = 1 # drop_off_type = 'no available' for the first station
            elif j == stop_num:
                pk_type = 1 # pickup_type = 'no available' for the final station


            # append to the new schedule st_tm 
            row = {'trip_id':tr_id,
                   'stop_sequence':j+1,
                   'stop_id':st,
                   'pickup_type':pk_type,
                   'drop_off_type':dr_type,
                   'arrival_time':time_base,
                   'departure_time':time_base,
                   'timepoint':0, # all default as 0, since we don't care timepoint for now
                   'stop_headsign':'IM A NEW ROUTE', # default, means nothing for us for now
                   'shape_dist_traveled':dist_base}
            st_tm.loc[i*stop_num+j]=row


    return st_tm


In [14]:
def add_route_stoptimes(stop_list = [],
              first_departure_time = '8:00:00',
              last_departure_time = '18:00:00',
              interval_time = 60, 
              stop_times_old = pd.DataFrame()):
    
    stop_times_old = convert_time_to_sec(stop_times_old)
    
    stop_seq = get_travel_time(stop_list,stop_times_old)
    
    st_tm = gen_new_schedule(stop_list = stop_list,
                     first_departure_time = first_departure_time,
                     last_departure_time = last_departure_timme,
                     interval_time = interval_time, 
                     stop_seq = stop_seq,
                     stop_times_old = stop_times_old)
    
    stop_times_updated = pd.merge(stop_times_old,st_tm, how = 'outer')
    
    return stop_times_updated
    

In [127]:
stop_times_filter = convert_time_to_sec(filter_stoptimes_by_routes(routes = [30,34,36]))
stop_times_filter.head()

index  trip_id  stop_sequence  stop_id  pickup_type  drop_off_type  \
0  36696   908292              1     7100            0              1   
1  36697   908292              2     7169            0              0   
2  36698   908292              3     9629            0              0   
3  36699   908292              4     9753            0              0   
4  36700   908292              5     9388            0              0   

   arrival_time  departure_time  timepoint         stop_headsign  \
0         22500           22500          1  E TOWNE: VIA NAKOOSA   
1         22544           22544          0  E TOWNE: VIA NAKOOSA   
2         22776           22776          0  E TOWNE: VIA NAKOOSA   
3         22800           22800          1  E TOWNE: VIA NAKOOSA   
4         22822           22822          0  E TOWNE: VIA NAKOOSA   

   shape_dist_traveled  
0               0.0092  
1               0.2630  
2               1.5965  
3               1.7287  
4               1.8701

In [129]:
stop_list_test = [9237,9175,9373,9918,9364,9100,9642,9378,9564,9802,9288,1420,1751,1309,1105,7100,7169,7167,7209,7633,9167,9811,9277,9243,9439,9137,9745,9951,9645,9291,9432,9719,9677,9800,9783,9391,9351,9140,9310,9620,9863,9917,9928,9422,9800]
stop_seq = get_travel_time(stop_list_test,stop_times)
stop_seq.head()

upstop_id  downstop_id  travel_time  travel_dist
0     9237.0       9175.0         33.0       0.1165
1     9175.0       9373.0         68.0       0.2397
2     9373.0       9918.0         43.0       0.1527
3     9918.0       9364.0         70.0       0.2461
4     9364.0       9100.0         86.0       0.2899

In [134]:
# input data
stop_list_test = [9237,9175,9373,9918,9364,9100,9642,9378,9564,9802,9288,1420,1751,1309,1105,7100,7169,7167,7209,7633,9167,9811,9277,9243,9439,9137,9745,9951,9645,9291,9432,9719,9677,9800,9783,9391,9351,9140,9310,9620,9863,9917,9928,9422,9800]
start_time = '8:00:00'
end_time = '18:00:00'
interval_time = 60 # minutes

# test function
new_stop_times = gen_new_schedule(stop_list = stop_list_test,
                                 first_departure_time = start_time,
                                 last_departure_time = end_time,
                                 interval_time = 90,
                                 stop_times_old = pd.read_csv(r"mmt_gtfs/trips.txt", sep = ",", engine = "python"))
new_stop_times.head()

trip_id  stop_sequence  stop_id  pickup_type  drop_off_type  \
0    990000.0            1.0   9237.0          0.0            1.0   
1    990000.0            2.0   9175.0          0.0            0.0   
2    990000.0            3.0   9373.0          0.0            0.0   
3    990000.0            4.0   9918.0          0.0            0.0   
4    990000.0            5.0   9364.0          0.0            0.0   
..        ...            ...      ...          ...            ...   
265  990005.0           41.0   9863.0          0.0            0.0   
266  990005.0           42.0   9917.0          0.0            0.0   
267  990005.0           43.0   9928.0          0.0            0.0   
268  990005.0           44.0   9422.0          0.0            0.0   
269  990005.0           45.0   9800.0          0.0            0.0   

     arrival_time  departure_time  timepoint   stop_headsign  \
0         28800.0         28800.0        0.0  IM A NEW ROUTE   
1         28833.0         28833.0        0.0  IM A NEW ROUTE   
2         28901.0         28901.0        0.0  IM A NEW ROUTE   
3         28944.0         28944.0        0.0  IM A NEW ROUTE   
4         29014.0         29014.0        0.0  IM A NEW ROUTE   
..            ...             ...        ...             ...   
265       57590.0         57590.0        0.0  IM A NEW ROUTE   
266       57669.0         57669.0        0.0  IM A NEW ROUTE   
267       57709.0         57709.0        0.0  IM A NEW ROUTE   
268       57759.0         57759.0        0.0  IM A NEW ROUTE   
269       57847.0         57847.0        0.0  IM A NEW ROUTE   

     shape_dist_traveled  
0                 0.0000  
1                 0.1165  
2                 0.3562  
3                 0.5089  
4                 0.7550  
..                   ...  
265               8.4197  
266               8.6546  
267               8.7745  
268               8.9244  
269               9.1739  

[270 rows x 10 columns]

In [15]:
# input data
stop_list_test = [9237,9175,9373,9918,9364,9100,9642,9378,9564,9802,9288,1420,1751,1309,1105,7100,7169,7167,7209,7633,9167,9811,9277,9243,9439,9137,9745,9951,9645,9291,9432,9719,9677,9800,9783,9391,9351,9140,9310,9620,9863,9917,9928,9422,9800]
start_time = '8:00:00'
end_time = '18:00:00'
interval_time = 60 # minutes

# test function
updated_stop_times = add_route_stoptimes(stop_list = stop_list_test,
                                 first_departure_time = start_time,
                                 last_departure_time = end_time,
                                 interval_time = 90,
                                 stop_times_old = pd.read_csv(r"mmt_gtfs/stop_times.txt", sep = ",", engine = "python"))
updated_stop_times

NameError: name 'last_departure_timme' is not defined